# Model Seq2Seq
attention based model
text -> Model -> new text

In [1]:
import tensorflow
import os
import pandas as pd
import yaml
from dataset import DataLoader
from dataset import DataPreprocessing
from dataset import TokenizerWrap
from model import AutoEncoderModel

In [3]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


2022-04-29 01:25:13.088577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-29 01:25:13.380775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:25:13.481405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:25:13.481665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [4]:
def config_loader(filepath):
    with open (filepath, 'r') as file_descriptor:
        config = yaml.safe_load(file_descriptor)
    return config

In [5]:
config = config_loader('config/en_de.yaml')

In [6]:
lang_src, lang_dest = config['language']['src'],config['language']['dest'] 

In [7]:
modelConfig_autoencoder = config_loader('config/autoencoder.yaml')

In [8]:
config

{'language': {'src': 'en', 'dest': 'de'},
 'dataset_path': '/europarl-v7.de-en.'}

In [9]:
lang_src

'en'

In [10]:
lang_dest

'de'

In [11]:
data_loader = DataLoader(config)

In [12]:
path = 'training'
os.listdir(path)

['europarl-v7.cs-en.cs',
 'europarl-v7.cs-en.en',
 'europarl-v7.de-en.de',
 'europarl-v7.de-en.en',
 'europarl-v7.es-en.en',
 'europarl-v7.es-en.es',
 'europarl-v7.fr-en.en',
 'europarl-v7.fr-en.fr',
 'questions_easy.csv']

In [13]:
src, dest = data_loader.loadData(path)

Loaded dataset at directory:
training/europarl-v7.de-en.en


In [14]:
type(src)

list

In [15]:
src[1]

'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'

In [16]:
dest[1]

'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.'

In [17]:
data_preprocessing = DataPreprocessing(config)

In [18]:
src = data_preprocessing.addStartEndPad(src)

In [19]:
src[0]

'aaaa Resumption of the session oooo'

In [20]:
df = data_preprocessing.createDataFrameSrcDest(src,dest)

In [21]:
df.head()

,en,de
0,aaaa Resumption of the session oooo,Wiederaufnahme der Sitzungsperiode
1,aaaa I declare resumed the session of the Euro...,"Ich erkläre die am Freitag, dem 17. Dezember u..."
2,"aaaa Although, as you will have seen, the drea...","Wie Sie feststellen konnten, ist der gefürchte..."
3,aaaa You have requested a debate on this subje...,Im Parlament besteht der Wunsch nach einer Aus...
4,"aaaa In the meantime, I should like to observe...",Heute möchte ich Sie bitten - das ist auch der...


In [22]:
df = df[:100000]

# Preprocessing

In [23]:
df.dtypes

en    object
de    object
dtype: object

In [24]:
df = df.astype(str)

In [25]:
df = data_preprocessing.toLowerCase(df)

In [26]:
df.head()

,en,de
0,aaaa resumption of the session oooo,wiederaufnahme der sitzungsperiode
1,aaaa i declare resumed the session of the euro...,"ich erkläre die am freitag, dem 17. dezember u..."
2,"aaaa although, as you will have seen, the drea...","wie sie feststellen konnten, ist der gefürchte..."
3,aaaa you have requested a debate on this subje...,im parlament besteht der wunsch nach einer aus...
4,"aaaa in the meantime, i should like to observe...",heute möchte ich sie bitten - das ist auch der...


In [27]:
from tqdm.notebook import tqdm

In [28]:
import numpy as np

In [29]:
num_words =10000
tokenizer_src = TokenizerWrap(texts=df[lang_src],
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

In [30]:
tokenizer_dest = TokenizerWrap(texts=df[lang_dest],
                               padding='post',
                               reverse=False,
                               num_words=num_words)

In [31]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(100000, 57)
(100000, 48)


In [32]:
tokens_src[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    3,  926,    1,    4,
       5176,    2], dtype=int32)

In [33]:
token_end = tokenizer_dest.word_index['weshalb']
token_end

978

In [34]:
idx = 2

In [35]:
tokens_src[idx]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    3, 4987,  106, 1610,   10, 1823,
        954,    4, 1450,    9, 2139,   68,    4,  207,    9,    7,   82,
          1,  180, 7944,    5, 1710,    1,  639,   22,   26,   44,   20,
        362,    2], dtype=int32)

In [36]:
tokenizer_src.tokens_to_string(tokens_src[idx])

'oooo dreadful were truly that disasters natural of series a suffered countries of number a in people the still materialise to failed the seen have will you as although aaaa'

In [37]:
df[lang_src][idx]

"aaaa although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. oooo"

In [38]:
encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape

(100000, 47)

In [39]:
tokens_dest[idx]

array([  30,   23,  755, 1034,   12,    2,   14, 4648,  100,   28,  150,
        923,  123,   76,  634,   10, 4326, 4250,  955,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [40]:
tokenizer_dest.tokens_to_string(tokens_dest[idx])

'wie sie feststellen konnten ist der nicht eingetreten doch sind bürger einiger unserer mitgliedstaaten opfer von schrecklichen naturkatastrophen geworden'

In [41]:
df[lang_dest][idx]


'wie sie feststellen konnten, ist der gefürchtete "millenium-bug " nicht eingetreten. doch sind bürger einiger unserer mitgliedstaaten opfer von schrecklichen naturkatastrophen geworden.'

In [42]:
from tensorflow.keras.layers import *

In [43]:

autoEncoder = AutoEncoderModel(modelConfig_autoencoder)
model_train = autoEncoder.BuildModel()

2022-04-29 01:26:04.017875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:26:04.018483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:26:04.018738: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:26:04.021399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-29 01:26:04.021722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not op

In [44]:
encoder_output, encoder_model = autoEncoder.encoderModel()
decoder_output, decoder_model = autoEncoder.decoderModel(encoder_output,alone=True)
        

In [45]:
import tensorflow as tf
tf.keras.utils.plot_model(
    encoder_model,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [46]:

tf.keras.utils.plot_model(
    decoder_model,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [47]:
tf.keras.utils.plot_model(
    model_train,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [48]:

callbacks = autoEncoder.getCallbacks()


In [49]:

try:
    model_train.load_weights(modelConfig_autoencoder['callbacks']['ModelCheckPoint']['filepath'])
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [50]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [51]:
y_data = \
{
    'decoder_output': decoder_output_data
}

In [52]:
validation_split = 10000 / len(encoder_input_data)
validation_split

0.1

In [53]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=128,
                epochs=1000,
                validation_split=validation_split,
                callbacks=callbacks)

Epoch 1/1000


2022-04-29 01:26:28.187821: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


704/704 [==============================] - ETA: 0s - loss: 1.7306
Epoch 1: val_loss improved from inf to 2.19738, saving model to CheckPoint
704/704 [==============================] - 171s 223ms/step - loss: 1.7306 - val_loss: 2.1974
Epoch 2/1000
704/704 [==============================] - ETA: 0s - loss: 1.6879
Epoch 2: val_loss improved from 2.19738 to 2.19234, saving model to CheckPoint
704/704 [==============================] - 155s 220ms/step - loss: 1.6879 - val_loss: 2.1923
Epoch 3/1000
 13/704 [..............................] - ETA: 2:25 - loss: 1.6147

KeyboardInterrupt: 

In [54]:
mark_end = ' oooo'
token_start = tokenizer_src.word_index[mark_end.strip()]
token_start

3

In [55]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = encoder_model.predict(input_tokens)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    
    decoder_input_data = np.zeros(shape=shape, dtype=int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = decoder_model.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

In [56]:
idx = 3
translate(input_text=df[lang_src][idx],
          true_output_text=df[lang_dest][idx])

1/1 [==============================] - 0s 52ms/step
Input text:
aaaa you have requested a debate on this subject in the course of the next few days, during this part-session. oooo

Translated text:
 schließlich haben wir heute morgen mit der aussprache über die debatte über die nächste sitzung zu diesem thema geführt                                                          

True output text:
im parlament besteht der wunsch nach einer aussprache im verlauf dieser sitzungsperiode in den nächsten tagen.



In [57]:
idx = 90
translate(input_text=df[lang_src][idx],
          true_output_text=df[lang_dest][idx])

1/1 [==============================] - 0s 40ms/step
Input text:
aaaa mr hänsch represented you on this occasion. oooo

Translated text:
 sie haben sie herr präsident                                                                                      

True output text:
der kollege hänsch hat sie dort vertreten.



In [58]:
translate(input_text="I want to join the European Union",
          true_output_text=None)

1/1 [==============================] - 0s 22ms/step
Input text:
I want to join the European Union

Translated text:
 das europäische parlament hat in der tat die möglichkeit                                                                              



In [59]:
tf.__version__


'2.10.0-dev20220427'

In [1]:
pip install pipreqs

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=46698ddd9c5b45a061b1ae7e496ed857ed3da3e87a238c9b4843b11bba5901b6
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pipreqs

In [11]:
!pipreqs --force

INFO: Successfully saved requirements file in /docker/requirements.txt


In [14]:
!pip install session_info

     |████████████████████████████████| 63 kB 951 kB/s eta 0:00:01
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8045 sha256=dd74c78314189f883dcab7cbd5be932d6e0e282e7a0369743cbe97dc89e2b95b
  Stored in directory: /root/.cache/pip/wheels/5c/1b/4d/111d73980c5c6a8e5e5905a19eccc34296cb359cac54c6c5b9
Successfully built session-info
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [17]:
import session_info
session_info.show()